In [1]:
import fastf1 as ff1
import pandas as pd
from datetime import datetime
import numpy as np
from fastf1.ergast import Ergast
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
ff1.Cache.enable_cache('data')

In [4]:
def get_driver_max_throttle_ratio(session, driver, max_throttle_threshold = 98):
    """
    Get the max throttle ratio for the fastest lap in the session for a selected driver.
    Parameters:
    - session - loaded session (i.e. session.load() must be called before)
    - driver - string, abbreviated driver name,
    - max_throttle_threshold - optional parameter, the threshold used to categorise the readout as "MAX THROTTLE".
      100 is not suggested due to readout errors. Default 98.

    Returns a dataframe:
    - driver
    - fastest lap max throttle ratio
    - tyre info (age, compound, fresh check)
    - weather info
      - rainfall rate - part of the laptime with the rain (values between 0 and 1)
      - avg track temp in Celcius
      - avg air temp in Celcuius
    """

    gp_name = session.event['EventName']
    race_weekend = session.event['Location']

    full_throttle = pd.DataFrame(columns = [
      'grand_prix',
      'location',
      'driver',
      'ratio',
      'compound',
      'tyre_age',
      'is_fresh_tyre',
      'avg_rainfall',
      'avg_track_temp',
      'avg_air_temp'])
    missing_info = pd.DataFrame(columns = [
      'grand_prix',
      'location',
      'driver'
      ])

    try:
      fastest_driver = session.laps.pick_driver(driver).pick_fastest()
      telemetry_driver = fastest_driver.get_telemetry().add_distance()
    
      #add weather info to the telemetry data
      telemetry_driver = pd.merge_asof(
          telemetry_driver,
          session.weather_data[['Time','Rainfall','TrackTemp', 'AirTemp']],
          left_on = 'SessionTime',
          right_on = 'Time'
          )

      #add info about the next/preious throttle input change
      telemetry_driver['nextThrottle'] = telemetry_driver.Throttle.shift(-1)
      telemetry_driver['previousThrottle'] = telemetry_driver.Throttle.shift(1)

      telemetry_driver_ltd = telemetry_driver.loc[
              (telemetry_driver.Throttle>=max_throttle_threshold)
              &((telemetry_driver.Throttle.shift(-1)<max_throttle_threshold)
                |((telemetry_driver.Throttle.shift(1)<max_throttle_threshold)
                  |(telemetry_driver.index.isin([telemetry_driver.index[0],telemetry_driver.index[-1]]))))
                  ].copy()

      #calculate the relative distance difference between portions of the track
      telemetry_driver_ltd['FTRelative'] = telemetry_driver_ltd.RelativeDistance - telemetry_driver_ltd.RelativeDistance.shift(1)

      telemetry_driver_ltd.FTRelative.fillna(0, inplace=True)

      #take every other row - i.e. include only rows with full throttle 
      ratio = telemetry_driver_ltd.loc[
          (telemetry_driver_ltd.nextThrottle<max_throttle_threshold)
          |(telemetry_driver_ltd.nextThrottle.isna())].FTRelative.sum()

      #create a dataframe with the results
      df = pd.DataFrame([{
         'grand_prix': gp_name,
         'location':race_weekend,
         'driver':driver,
         'ratio':ratio,
          'compound':fastest_driver['Compound'],
          'tyre_age':fastest_driver['TyreLife'],
          'is_fresh_tyre':fastest_driver['FreshTyre'],
          'avg_rainfall':telemetry_driver['Rainfall'].mean(),
          'avg_track_temp':telemetry_driver['TrackTemp'].mean(),
          'avg_air_temp':telemetry_driver['AirTemp'].mean()
          }])
      
      full_throttle = pd.concat([full_throttle, df], ignore_index=True, axis = 0)
      
    except KeyError:
      # in some cases we do not have the telemetry data due to tech issues.
      # this dataframe is created to keep a note about that.
      df_dictionary = pd.DataFrame([{
         'grand_prix':gp_name,
         'location':race_weekend,
         'driver':driver
         }])
      missing_info = pd.concat([missing_info, df_dictionary], ignore_index=True, axis = 0)

    return full_throttle,missing_info

In [5]:
def get_all_drivers_throttle_input(session):
    """
    Get the max throttle driven single rap ratio for all drivers.
    Params:
    - session - race weekend session (session.load() must be called before)
    Returns
    - correct_readings dataframe containing:
        - year
        - race weekend name
        - session id
        - driver
        - single lap max throttle ratio
    - missing_info dataframe containing drivers with missing information reg. throttle input
    """
    drivers = pd.unique(session.laps['Driver'])

    gp_name = session.event['EventName']
    race_weekend = session.event['Location']

    full_throttle = pd.DataFrame()
    missing_info = pd.DataFrame()

    for driver in drivers:
        #print(driver)
        driver_info, missing_data = get_driver_max_throttle_ratio(session, driver)
        driver_info['grand_prix'] = gp_name
        driver_info['location'] = race_weekend
        full_throttle = pd.concat([full_throttle, driver_info], ignore_index=True, axis = 0)
            
       
        missing_info = pd.concat([missing_info, missing_data], ignore_index=True, axis = 0)

    # removing obvious incorrect readings
    # there is not a single track with max throttle ratio > 85% or < 40%
    full_throttle.loc[(full_throttle.ratio>0.85)|(full_throttle.ratio<0.4), 'ratio'] = np.NaN

    correct_readings = full_throttle.loc[~full_throttle.ratio.isna()]
    incorrect_readings = full_throttle.loc[full_throttle.ratio.isna()]

    missing_info = pd.concat([missing_info, incorrect_readings[['grand_prix','location','driver']]], ignore_index=True, axis = 0)

    return correct_readings, missing_info
    

In [51]:
def get_session_data(year,race_weekend, session_name):
    """
    Get throttle input data for the whole session (best lap per driver).
    Parameters:
    - year - int
    - race_weekend - string, location of the race (city; f.e. Suzuka, Sakhir, Jeddah, Melbourne, Baku)
    - session_name - string
        - FP1/FP2/FP3 - Free Practice 1/2/3
        - SQ - Sprint Qualifying
        - SS - Sprint Shootout
        - R - Race
    Returns:
    - full_throttle dataframe containing throttle input data for all drivers in a single session
    - missing_info dataframe containing the information about the drivers with missing throttle input data
    """
    session = ff1.get_session(year,race_weekend,session_name)
    session.load()
    full_throttle, missing_info = get_all_drivers_throttle_input(session)

    full_throttle['session'] = session_name
    missing_info['session'] = session_name

    return full_throttle, missing_info

In [69]:
def get_track_full_throttle(season, most_recent_practice = True):
    """
    Get the info about all races to date in the current Formula 1 season.
    Currently covering:
    - sprint race weekends - Free Practice 1 
    - conventional weekends - Free Practice 1 & 2
    
    Parameters:
    - season - int indicating the year
    - most_recent_practice - boolean, checking whether all season data is required (False) or just the last one is needed (True - Default)

    Returns 2 dataframes:
    - throttle input df - containing all information about the fastest lap incl. the throttle input, weather and tyres
    - missing info df - containing all cases of missing or incorrect telemetry
    """

    schedule = ff1.get_event_schedule(season)
    season_races = schedule.loc[
        #every race weekend up BEFORE today
        (schedule.Session1DateUtc < datetime.utcnow())&
        #excluding testing
        (schedule.EventFormat!='testing')]
    
    #using the parameter to get either all races in the season or just the last one.
    if most_recent_practice: 
        #only most recent
        races = season_races.loc[(season_races.RoundNumber==season_races.RoundNumber.max())]
    else:
        #all races until now
        races = season_races.copy()

    throttle_df = pd.DataFrame()
    missing_df = pd.DataFrame()

    conventional_sessions = ['FP1','FP2','FP3']

    for race in races.Location:
        print(race)

        if races.loc[races['Location']==race, 'EventFormat'].to_list()[0]=='conventional':
            #if a race is conventional - get FP1+FP2+FP3 -> all sessions before quali
            for session in conventional_sessions:
                full_throttle, missing_info = get_session_data(season, race, session)
                throttle_df = pd.concat([throttle_df, full_throttle], ignore_index=True, axis = 0)
                missing_df = pd.concat([missing_df, missing_info], ignore_index=True, axis = 0)
        else:
             #if a race is sprint format - get FP1 -> all sessions before quali
            full_throttle, missing_info = get_session_data(season, race, 'FP1')
            throttle_df = pd.concat([throttle_df, full_throttle], ignore_index=True, axis = 0)
            missing_df = pd.concat([missing_df, missing_info], ignore_index=True, axis = 0)

    return throttle_df, missing_df

In [78]:
def get_elevation(latitude, longitude):
    """ 
    Get the elevation of a single point based on coordinates.
    
    Parameters:
    - latitude
    - longitude

    Returns single value - altitude above sea level in meters.
    """
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={latitude},{longitude}"
    
    response = requests.get(url)
    data = response.json()
    elevation = data["results"][0]["elevation"]

    return elevation


In [79]:
def get_circuits(season):
    """ 
    Get main geolocation info of the tracks:
    - latitude
    - longitude
    - official circuit name
    - altitude above sea level

    Parameters:
    - season - int indicating the year/season of Formula 1

    Returns
    - dataframe containing all info about the track location.
    """

    ergast = Ergast()

    racetracks = ergast.get_circuits(season)

    altitudes = pd.DataFrame()

    for racetrack in racetracks.circuitName:
        latitude = racetracks.loc[racetracks.circuitName==racetrack].iat[0,3]
        longitude = racetracks.loc[racetracks.circuitName==racetrack].iat[0,4]
        altitude = get_elevation(latitude,longitude)
        
        df = pd.DataFrame([{
            'circuitName': racetrack,
            'altitude': altitude,
            }])
        
        altitudes = pd.concat([altitudes, df], ignore_index=True, axis = 0)

    racetracks = racetracks.merge(altitudes,how = 'left',on='circuitName')

    return racetracks

# Work in progress

1.  slow/medium/fast corners
    - Chickanes
2. quali pace
3. race pace
4. breaking
5. when to query the data?
    - the time of the race is not consistent
    - on sprint race weekends the quali takes place on Friday
    - US/Asia/Australia races happen at different times

In [83]:
schedule.loc[schedule.EventFormat!='testing',

        ['RoundNumber',
          'Country',
          'Location',
          'EventName',
          'EventFormat',
          'Session1',
          'Session1DateUtc',
          'Session2',
          'Session2DateUtc',
          'Session3',
          'Session3DateUtc',
          'Session4',
          'Session4DateUtc',
          'Session5',
          'Session5DateUtc',
          ]]

,RoundNumber,Country,Location,EventName,EventFormat,Session1,Session1DateUtc,Session2,Session2DateUtc,Session3,Session3DateUtc,Session4,Session4DateUtc,Session5,Session5DateUtc
1,1,Bahrain,Sakhir,Bahrain Grand Prix,conventional,Practice 1,2023-03-03 11:30:00,Practice 2,2023-03-03 15:00:00,Practice 3,2023-03-04 11:30:00,Qualifying,2023-03-04 15:00:00,Race,2023-03-05 15:00:00
2,2,Saudi Arabia,Jeddah,Saudi Arabian Grand Prix,conventional,Practice 1,2023-03-17 13:30:00,Practice 2,2023-03-17 17:00:00,Practice 3,2023-03-18 13:30:00,Qualifying,2023-03-18 17:00:00,Race,2023-03-19 17:00:00
3,3,Australia,Melbourne,Australian Grand Prix,conventional,Practice 1,2023-03-31 02:30:00,Practice 2,2023-03-31 06:00:00,Practice 3,2023-04-01 02:30:00,Qualifying,2023-04-01 06:00:00,Race,2023-04-02 05:00:00
4,4,Azerbaijan,Baku,Azerbaijan Grand Prix,sprint_shootout,Practice 1,2023-04-28 09:30:00,Qualifying,2023-04-28 13:00:00,Sprint Shootout,2023-04-29 08:30:00,Sprint,2023-04-29 13:30:00,Race,2023-04-30 11:00:00
5,5,United States,Miami,Miami Grand Prix,conventional,Practice 1,2023-05-05 18:00:00,Practice 2,2023-05-05 21:30:00,Practice 3,2023-05-06 16:30:00,Qualifying,2023-05-06 20:00:00,Race,2023-05-07 19:30:00
6,6,Monaco,Monaco,Monaco Grand Prix,conventional,Practice 1,2023-05-26 11:30:00,Practice 2,2023-05-26 15:00:00,Practice 3,2023-05-27 10:30:00,Qualifying,2023-05-27 14:00:00,Race,2023-05-28 13:00:00
7,7,Spain,Barcelona,Spanish Grand Prix,conventional,Practice 1,2023-06-02 11:30:00,Practice 2,2023-06-02 15:00:00,Practice 3,2023-06-03 10:30:00,Qualifying,2023-06-03 14:00:00,Race,2023-06-04 13:00:00
8,8,Canada,Montréal,Canadian Grand Prix,conventional,Practice 1,2023-06-16 17:30:00,Practice 2,2023-06-16 20:30:00,Practice 3,2023-06-17 16:30:00,Qualifying,2023-06-17 20:00:00,Race,2023-06-18 18:00:00
9,9,Austria,Spielberg,Austrian Grand Prix,sprint_shootout,Practice 1,2023-06-30 11:30:00,Qualifying,2023-06-30 15:00:00,Sprint Shootout,2023-07-01 10:00:00,Sprint,2023-07-01 14:30:00,Race,2023-07-02 13:00:00
10,10,Great Britain,Silverstone,British Grand Prix,conventional,Practice 1,2023-07-07 11:30:00,Practice 2,2023-07-07 15:00:00,Practice 3,2023-07-08 10:30:00,Qualifying,2023-07-08 14:00:00,Race,2023-07-09 14:00:00


In [84]:
def create_corners_info(location, slow, medium, fast):
    return pd.DataFrame([{
        'Location': location,
        'slow':slow,
        'medium':medium,
        'fast':fast
        }])

def create_pirelli_tyre_info(location, traction, evolution, lateral_load, abrasion, braking, grip, tyre_stress, downforce):
    return pd.DataFrame([{
        'Location': location,
        'traction': traction,
        'evolution': evolution,
        'lateral_load': lateral_load,
        'abrasion': abrasion,
        'braking': braking,
        'grip': grip,
        'tyre_stress': tyre_stress,
        'downforce':downforce
        }])


In [ ]:
bahrain_corners = create_corners_info('Sakhir',4,4,7)
jeddah_corners = create_corners_info('Jeddah',4,4,7)
melbourne_corners = create_corners_info('Melbourne', 3, 5, 6)
baku_corners = create_corners_info('Baku',11,3,6)
miami_corners = create_corners_info('Miami', 7,5,7)
monaco_corners = create_corners_info('Monaco', 9,5,5)
barcelona_corners = create_corners_info('Barcelona', 2,6,6) #2023
barcelona_corners = create_corners_info('Barcelona', 4,8,4) #2022
montreal_corners = create_corners_info('Montreal', 3,8,3)
spielberg_corners = create_corners_info('Spielberg', 2,3,5)
silverstone_corners = create_corners_info('Silverstone', 4,4,10)
budapest_corners = create_corners_info('Budapest', 3,7,4)
spa_corners = create_corners_info('Spa', 3,8,8)
imola_corners = create_corners_info('Imola', 5,3,11)
abu_dhabi_pirelli = create_corners_info('Abu Dhabi',3,6,7)
las_vegas_corners = create_corners_info('Las Vegas', 7,4,6)
mexico_corners = create_corners_info('Mexico City', 7,6,4)
sao_paulo_corners = create_corners_info('Sao Paulo', 1,6,8)
suzuka_corners = create_corners_info('Suzuka', 3,4,11)
qatar_corners = create_corners_info('Qatar', 1,4,11)
austin_corners = create_corners_info('Austin', 7,5,8)
zandvoort_corners = create_corners_info('Zandvoort', 2,6,6)
monza_corners = create_corners_info('Monza', 2,5,4)
singapore_corners = create_corners_info('Singapore', 10,8,5) #2022
singapore_corners = create_corners_info('Singapore', 7,7,5) #2023

corners = pd.concat([bahrain_corners,
           jeddah_corners,
           melbourne_corners,
           baku_corners,
           miami_corners,
           monaco_corners,
           barcelona_corners,
           montreal_corners,
           spielberg_corners,
           silverstone_corners ,
           budapest_corners ,
           spa_corners ,
           imola_corners ,
           abu_dhabi_pirelli ,
           las_vegas_corners ,
           mexico_corners,
           sao_paulo_corners ,
           suzuka_corners ,
           qatar_corners ,
           austin_corners,
           zandvoort_corners,
           monza_corners,
           singapore_corners,
           ], 
           ignore_index=True
           )


In [ ]:
bahrain_pirelli = create_pirelli_tyre_info('Sakhir', 4, 4, 3, 5, 4, 3, 3, 3)
jeddah_pirelli = create_pirelli_tyre_info('Jeddah', 2, 3, 4, 2, 2, 3, 3, 2)
melbourne_pirelli = create_pirelli_tyre_info('Melbourne', 2, 4, 3, 2, 2, 3, 3, 3)
baku_pirelli = create_pirelli_tyre_info('Baku', 5, 5, 1, 1, 4, 1, 3, 1)
miami_pirelli = create_pirelli_tyre_info('Miami',3,5,3,2,3,3,3,2)
monaco_pirelli = create_pirelli_tyre_info('Monaco', 5,5,1,1,2,1,1,5)
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,5,4,3,3,5,4) #2023
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,4,4,3,3,4,4) #2022
montreal_pirelli = create_pirelli_tyre_info('Montreal', 5,5,1,2,5,1,3,1)

spielberg_pirelli = create_pirelli_tyre_info('Spielberg', 2,3,5)
silverstone_pirelli = create_pirelli_tyre_info('Silverstone', 4,4,10)
budapest_pirelli = create_pirelli_tyre_info('Budapest', 3,7,4)
spa_pirelli = create_pirelli_tyre_info('Spa', 3,8,8)
imola_pirelli = create_pirelli_tyre_info('Imola', 5,3,11)
abu_dhabi_pirelli = create_pirelli_tyre_info('Abu Dhabi',3,6,7)
las_vegas_pirelli = create_pirelli_tyre_info('Las Vegas', 7,4,6)
mexico_pirelli = create_pirelli_tyre_info('Mexico City', 7,6,4)
sao_paulo_pirelli = create_pirelli_tyre_info('Sao Paulo', 1,6,8)
suzuka_pirelli = create_pirelli_tyre_info('Suzuka', 3,4,11)
qatar_pirelli = create_pirelli_tyre_info('Qatar', 1,4,11)
austin_pirelli = create_pirelli_tyre_info('Austin', 7,5,8)
zandvoort_pirelli = create_pirelli_tyre_info('Zandvoort', 2,6,6)
monza_pirelli = create_pirelli_tyre_info('Monza', 2,5,4)
singapore_pirelli = create_pirelli_tyre_info('Singapore', 10,8,5) #2022
singapore_pirelli = create_pirelli_tyre_info('Singapore', 7,7,5) #2023


In [ ]:
pirelli = pd.concat([
    bahrain_pirelli,
    jeddah_pirelli,
    melbourne_pirelli,
    baku_pirelli,
    miami_pirelli ,
    monaco_pirelli ,
    barcelona_pirelli,
    montreal_pirelli ,
    ], 
    ignore_index=True
    )